# QC protocol for Private Weather Stations

This notebook presents how to use the Python package `pypwsqc`, a quality assurance protocol developed for automated private weather stations (PWS). The protocol consists of three filters; the Faulty Zero filter, the High Influx filter and the Station Outlier filter.

The package is based on the original R code available at https://github.com/LottedeVos/PWSQC/.

Publication: de Vos, L. W., Leijnse, H., Overeem, A., & Uijlenhoet, R. (2019). Quality control for crowdsourced personal weather stations to enable operational rainfall monitoring. Geophysical Research Letters, 46(15), 8820-8829

`pypwsqc` depends on the `poligrain`, `xarray`, `pandas` and `numpy` packages. Make sure to install and import the required packages first.

In [1]:
import poligrain as plg
import xarray as xr
import numpy as np
import pandas as pd

# import pypwsqc

In [2]:
import matplotlib.pyplot as plt

In [3]:
ds_pws = xr.open_dataset('C:/Users/a002461/OPENSENSE/data/OpenSense_PWS_data_HI_FZ_filtered_rainfall.nc')

# slice to one month
#ds_pws = ds_pws.sel(time = slice('2017-07-01','2017-07-31'))

In [4]:
ds_pws

<xarray.Dataset>
Dimensions:               (time: 219168, id: 134)
Coordinates:
  * time                  (time) datetime64[ns] 2016-05-01T00:05:00 ... 2018-...
  * id                    (id) <U6 'ams1' 'ams2' 'ams3' ... 'ams133' 'ams134'
    elevation             (id) <U3 ...
    latitude              (id) float64 ...
    longitude             (id) float64 ...
    x                     (id) float64 ...
    y                     (id) float64 ...
Data variables:
    rainfall              (id, time) float64 ...
    nbrs_not_nan          (id, time) int32 ...
    reference             (id, time) float64 ...
    fz_flag               (id, time) int32 ...
    hi_flag               (id, time) int32 ...
    fz_filtered_rainfall  (id, time) float64 ...
    hi_filtered_rainfall  (id, time) float64 ...
Attributes:
    title:                 PWS data from Amsterdam
    file author:           Maximilian Graf
    institution:           Wageningen University and Research, Department of ...
    date:                  2022-10-18 10:32:00
    source:                Netamo PWS
    history:               Data derived and reformated from the originally pu...
    naming convention:     OpenSense-0.1
    license restrictions:  CC-BY 4.0 https://creativecommons.org/licenses/by/...
    reference:             https://doi.org/10.1029/2019GL083731
    comment:

## Download example data

In this example, we use an open PWS dataset from Amsterdam, called the "AMS PWS" dataset. By running the cell below, an example NetCDF-file will be downloaded to your current repository (if your machine is connected to the internet).

In [ ]:
!curl -OL https://github.com/OpenSenseAction/OS_data_format_conventions/raw/main/notebooks/data/OpenSense_PWS_example_format_data.nc

## Data preparations

This package handles rainfall data as `xarray`  Datasets. The data set must have `time` and `id` dimensions, `latitude` and `longitude` as coordinates, and `rainfall` as data variable.

An example of how to convert .csv data to a `xarray` dataset is found [here](https://github.com/OpenSenseAction/OS_data_format_conventions/blob/main/notebooks/PWS_example_dataset.ipynb).

We now load the data set under the name  `ds_pws`.

In [ ]:
ds_pws = xr.open_dataset("OpenSense_PWS_example_format_data.nc")
ds_pws

### Reproject coordinates 

First we reproject the coordinates to a local metric coordinate reference system to allow for distance calculations. In the Amsterdam example we use EPSG:25832. **Remember to use a local metric reference system for your use case!** We use the function `spatial.project_point_coordinates` in the `poligrain`package. 

In [5]:
ds_pws.coords["x"], ds_pws.coords["y"] = plg.spatial.project_point_coordinates(
    x=ds_pws.longitude, y=ds_pws.latitude, target_projection="EPSG:25832"
)

### Create distance matrix

Then, we calculate the distances between all stations in our data set. If your data set has a large number of stations this can take some time.

In [6]:
distance_matrix = plg.spatial.calc_point_to_point_distances(ds_pws, ds_pws)

### Calculate data variables 

Next, we will calculate the data variables `nbrs_not_nan` and `reference` that are needed to perform the quality control. If you have already processed your data with the FZ and HI-filters, your `xarray` data set already have these variables and you can proceed to the next section.

`nbrs_not_nan`:
Number of neighbours within a specificed range `max_distance` around the station that are reporting rainfall for each time step. The selected range depends on the use case and area of interest. In this example we use 10'000 meters. 

 `reference`:
Median rainfall of all stations within range `max_distance` from each station.

`max_distance` is called 'd' in the original publication.

### Select considered range around each station

In [7]:
max_distance = 10e3

In [ ]:
%%time
ds_pws = ds_pws.load()

nbrs_not_nan = []
reference = []

for pws_id in ds_pws.id.data:
    neighbor_ids = distance_matrix.id.data[
        (distance_matrix.sel(id=pws_id) < max_distance)
        & (distance_matrix.sel(id=pws_id) > 0)
    ]

    N = ds_pws.rainfall.sel(id=neighbor_ids).notnull().sum(dim="id")
    nbrs_not_nan.append(N)

    median = ds_pws.sel(id=neighbor_ids).rainfall.median(dim="id")
    reference.append(median)

ds_pws["nbrs_not_nan"] = xr.concat(nbrs_not_nan, dim="id")
ds_pws["reference"] = xr.concat(reference, dim="id")

### Initialize data variables

We initialize data variables for the resulting SO-flags and the median pearson correlation with neighboring stations with the value -999. If the variables have the value 0 (passed the test), 1 (did not pass the test) or -1 (not enough information) after running the SO-filter, we know that these time series have been evaluated. If the value is still -999, this means that something went wrong as the data has not been processed. 

We also save the threshold `gamma` as a variable. In this way we can easily visualize if the median correlation with neighbors drops below this threshold, which is the condition for raising a SO-flag (see below).

In [17]:
gamma = 0.17 # 0.15 

In [18]:
ds_pws['so_flag'] = xr.DataArray(np.ones((len(ds_pws.id), len(ds_pws.time)))*-999, dims=("id", "time"))
ds_pws['median_corr_nbrs'] = xr.DataArray(np.ones((len(ds_pws.id), len(ds_pws.time)))*-999, dims=("id", "time"))
ds_pws['gamma'] = xr.DataArray(np.ones((len(ds_pws.id), len(ds_pws.time)))*gamma, dims=("id", "time"))

The private weather stations in your data set likely have different installation dates, and therefore different starting days of their time series. The `time` dimension of the `xarray` data set will start on the time and day that the first station was installed. Stations that were installed later will have `NaN` until their installation date. As we obviously cannot perform quality control on a station that has been installed yet, we set the SO-flag to -1 up to the first valid rainfall observation for each station. 

In [31]:
# For each station (ID), get the index of the first non-NaN rainfall value
first_non_nan_index = ds_pws["rainfall"].notnull().argmax(dim="time")

## Quality control

Now the data set is prepared to run the quality control.

### Apply Station Outlier filter

Conditions for raising Station Outlier flag:

* Median of the rolling pearson correlation with all neighboring stations within range `max_distance` is less than threshold `gamma`
* Filter cannot be applied if less than `nstat` neighbours are reporting data (SO flag is set to -1)
*  **Filter cannot be applied if there are less than `nstat` neighbours with less than `mmatch` intervals overlapping with the evaluated station(SO flag is set to -1)**

For settings for parameter `mint`, `mmatch`, `gamma`, and `nstat`, see table 1 in https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019GL083731 

Note! The SO-filter is different compared with the original R-code. In its original implementation, any interval with at least `mrain` intervals of nonzero rainfall measurements is evaluated. In this implementation, only a fixed rolling window of `mint` intervals is evaluated. Therefore, the `mrain` from the orignal code is not needed. In the original publication, the variable `mint` (the evaluation period) is set to 4032. For 5-minute data, this is equivalent of two weeks. When the option of a variable evaluation period is excluded, two weeks is often too short as there might not be enough wet periods in the last two weeks to calculate the correlation. This results in a lot of '-1'-flags (filter cannot be applied). It is suggested to use a longer evaluation period, for example four weeks (`mint` = 8064 for 5-minute data).

`max_distance` is called 'd' in the original publication.


In [10]:
mint = 8064  
mmatch = 100
gamma = 0.17 # 0.15
beta = 0.2
n_stat = 5
max_distance = 10e3 

## move cells below to flagging.py

In [11]:
def so_filter(da_station, da_neighbors, window_length):

    # rolling pearson correlation
    s_station = da_station.to_series()
    s_neighbors = da_neighbors.to_series()
    corr = s_station.rolling(window_length, min_periods= 1).corr(s_neighbors)
    ds = xr.Dataset.from_dataframe(pd.DataFrame({'corr': corr}))

    # create dataframe of neighboring stations
    df = da_neighbors.to_dataframe()
    df = df["rainfall"].unstack("id")

    # boolean arrays - True if a rainy time step, False if 0 or NaN.
    rainy_timestep_at_nbrs = (df > 0)
    
    # rolling sum of number of rainy timesteps in last mint period, per neighbor. 
    wet_timesteps_last_mint_period = rainy_timestep_at_nbrs.rolling(mint, min_periods=1).sum()
    
    # per time step and neighbor, does the nbr have more than mmatch wet time steps in the last mint period? (true/false)
    enough_matches_per_nbr = (wet_timesteps_last_mint_period > mmatch)
    
    # summing how many neighbors that have enough matches per time step
    nr_nbrs_with_enough_matches = enough_matches_per_nbr.sum(axis = 1)

    ds['matches'] = xr.DataArray.from_series(nr_nbrs_with_enough_matches)
    
    return ds

In [42]:
%%time

for i in [16]: # range(len(ds_pws.id)):
    
    # create data set for station i
    ds_station = ds_pws.isel(id=i) 
    pws_id = ds_station.id.values

    # picking stations within max_distnance, excluding itself, for the whole duration of the time series
    neighbor_ids = distance_matrix.id.data[(distance_matrix.sel(id=pws_id) < max_distance) & (distance_matrix.sel(id=pws_id) > 0)]

    #create data set for neighbors
    ds_neighbors = ds_pws.sel(id=neighbor_ids)

    # if there are no observations in the time series, filter cannot be applied to the whole time series
    if ds_pws.rainfall.sel(id=pws_id).isnull().all():
        ds_pws.so_flag[i, :] = -1
        ds_pws.median_corr_nbrs[i,:] = -1
        continue

    # if there are not enough stations nearby, filter cannot be applied to the whole time series
    elif (len(neighbor_ids) < n_stat):
        ds_pws.so_flag[i, :] = -1
        ds_pws.median_corr_nbrs[i,:] = -1
        continue 
        
    else: 

    # run so-filter
        ds_so_filter = so_filter(ds_station.rainfall, ds_neighbors.rainfall, window_length=mint)

        median_correlation = ds_so_filter.corr.median(dim='id', skipna = True)
        ds_pws.median_corr_nbrs[i] = median_correlation
        
        so_array = (median_correlation < gamma).astype(int)
        
    # filter can not be applied if less than n_stat neighbors have enough matches
        ds_pws.so_flag[i] = xr.where(ds_so_filter.matches < n_stat, -1, so_array)

    # Set so_flag to -1 up to first valid index
        first_valid_time = first_non_nan_index[i].item()
        ds_pws["so_flag"][i, :first_valid_time] = -1 

    # disregard warm up period
        ds_pws.so_flag[i, first_valid_time:(first_valid_time+mint)] = -1
# ds_pws.to_netcdf('C:/Users/a002461/OPENSENSE/data/fourth_SO_flags_test.nc')

CPU times: total: 32.8 s
Wall time: 33.4 s


In [ ]:
so_flag = pypwsqc.flagging.so_filter(
    ds_pws,
    distance_matrix,
    mint = 4032,
    mmatch = 200,
    gamma = 0.15,
    n_stat = 5,
    max_distance = 10e3,
)

In [19]:
#so_flag = pypwsqc.flagging.so_filter(
#    ds_pws.rainfall,
#    ds_pws.nbrs_not_nan,
#    ds_pws.reference,
#    distance_matrix,
#    mint = 4032,
#    mrain = 100,
#    mmatch = 200,
#    gamma = 0.15,
#   beta = 0.2,
#    n_stat = 5,
#    max_distance = 10e3,
#)